# MedHelm data collection

## Downloading the data

This notebook requires you to already have downloaded the data you want to transform. Follow [Helm's guide to download the raw results.](https://crfm-helm.readthedocs.io/en/stable/downloading_raw_results/).

You can also manually explore the [data available in the browser](https://console.cloud.google.com/storage/browser/crfm-helm-public/medhelm?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&inv=1&invt=AbtJ_g).

In [1]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), "../"))


## MedHelm data info

In [10]:
MEDHELM_DATA_PATH = "../../data/medhelm/benchmark_output"
MEDHELM_RUN_PATH = os.path.join(MEDHELM_DATA_PATH, "runs/v1.0.0")

OUTPUT_PATH = "../../data/medhelm/medbench_connection/medhelm-fixed"

if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

Add below the datasets you want to collect data from.

❗If you want to collect and transform the actual dataset and model outputs, only text-based datasets are supported by the transformation logic.

💡For metrics collection, however, any dataset should work. Although the `model_run` will be incomplete, as it won't include images.

In [11]:
TARGET_DATASETS = [
    "aci_bench",
    "ehr_sql",
    "head_qa",
    "medbullets",
    "medcalc_bench",
    "medec",
    "medhallu",
    # "medi_qa",
    "medication_qa",
    # "mimiciv_billing_code",
    "mtsamples_procedures",
    "mtsamples_replicate",
    "pubmed_qa",
    "race_based_med",
    # "med_dialog",
]

Mark multiple choice datasets, as their references needs mapping

In [12]:
MULTIPLE_CHOICE_DATASETS = [
    "head_qa",
    "medbullet", # There's a typo in the helm data.
    "pubmed_qa",
]

Generally, Helm uses different names for their metrics. All of the MedHelm metrics will be kept with their original name, however we also duplicate specific ones so that Arena can display them.

In [13]:
METRICS_MAP = {
    "BERTScore-F": "bert_score",
    "rouge_1": "rouge1",
    "rouge_2": "rouge2",
    "rouge_l": "rougeL",
}

Helm splits stores all their data in the files defined below.

In [14]:
RUN_METADATA_FILENAME: str = "run_spec.json"

# Contains `results` and `instance`
RUN_RESULTS_FILENAME: str = "scenario_state.json"

DATASET_METADATA_FILENAME: str = "scenario.json"
DATASET_INSTANCES_FILENAME: str = "instances.json"

DATASET_METRICS_FILENAME: str = "stats.json"
INDIVIDUAL_METRICS_FILENAME: str = "per_instance_stats.json"

Below is a mapping of `medbench` data classes to `medhelm`.

```yaml
ModelRun:
    Model:
        name: 
            - run_spec.adapter_spec.model
            - scenario_state.adapter_spec.model
        version:

    Dataset:
        name: scenario.name
        description: scenario.description
        instances:
            - instances
            - scenario_state.request_states[].instance
    
    ModelOutput[]: scenario_state.request_states[].result
```

## Collect and transform MedHelm data.

In [8]:
import json
from datetime import datetime, timezone

from medbench.datasets import (
    CORRECT_TAG,
    Data,
    Dataset,
    Instance,
    MediaObject,
    Reference,
)
from medbench.models import Model, ModelOutput, ModelRun

Helper functions for transforming helm run and metrics into medbench format

In [ ]:
def process_helm_run(helm_run_path: str) -> ModelRun:
    # Open Helm run files
    run_metadata: dict
    with open(
        os.path.join(helm_run_path, RUN_METADATA_FILENAME), "r"
    ) as run_metadata_file:
        run_metadata = json.load(run_metadata_file)

    dataset_metadata: dict
    with open(
        os.path.join(helm_run_path, DATASET_METADATA_FILENAME), "r"
    ) as dataset_metadata_file:
        dataset_metadata = json.load(dataset_metadata_file)

    run_results: dict
    with open(
        os.path.join(helm_run_path, RUN_RESULTS_FILENAME), "r"
    ) as run_results_file:
        run_results = json.load(run_results_file)

    # Initialize medbench objects
    model = Model(
        name=run_metadata["adapter_spec"]["model"],
        version="",
    )

    dataset = Dataset(
        name=dataset_metadata["name"],
        description=dataset_metadata["description"],
        instances=[],
    )

    model_run = ModelRun(
        id=run_metadata["name"],
        model=model,
        dataset=dataset,
        results=[],
    )

    # Process instances and results (from scenario_state.json)
    for request_state in run_results["request_states"]:
        instance_data = request_state["instance"]

        references = []
        for i, reference_data in enumerate(instance_data["references"]):
            metadata: dict = None

            if dataset.name in MULTIPLE_CHOICE_DATASETS:
                MULTIPLE_CHOICE_LETTERS = "ABCDEFGH"
                metadata = {"multiple_choice_letter": MULTIPLE_CHOICE_LETTERS[i]}

            data = Data.from_text(
                data=reference_data["output"]["text"], metadata=metadata
            )

            references.append(
                Reference(
                    output=data,
                    tags=[CORRECT_TAG] if "correct" in reference_data["tags"] else [],
                )
            )

        instance = Instance(
            id=instance_data["id"],
            input=Data.from_text(data=instance_data["input"]["text"]),
            references=references,
            split=instance_data["split"],
            sub_split="",
            perturbation="",
            metadata="",
        )
        dataset.instances.append(instance)

        result_data = request_state["result"]
        result = ModelOutput(
            input_id=instance_data["id"],
            completions=Data.from_text(data=result_data["completions"][0]["text"]),
        )

        if not result_data["success"]:
            result.finish_reason = result_data["error"]
            result.error = result_data["error"]

        model_run.results.append(result)

    return model_run

In [194]:
def transform_helm_metrics(
    stats: dict,
    aggregation_method: str,
    target_split: str,
    metrics_map: dict,
) -> dict:
    metrics = {}
    for stat in stats:
        if stat["name"]["split"] != target_split:
            continue
        
        if aggregation_method not in stat:
            continue
        
        metrics[stat["name"]["name"]] = stat[aggregation_method]

    for helm_metric in metrics_map:
        if helm_metric in metrics:
            metrics[metrics_map[helm_metric]] = metrics[helm_metric]

    return metrics

def process_helm_metrics(
    helm_run_path: str,
    aggregation_method: str = "mean",
    target_split: str = "test",
    metrics_map: dict = METRICS_MAP,
) -> dict:
    # Open Helm metrics files
    aggregated_metrics_data: dict
    with open(
        os.path.join(helm_run_path, DATASET_METRICS_FILENAME), "r"
    ) as metrics_file:
        aggregated_metrics_data = json.load(metrics_file)

    instance_level_metrics_data: dict
    with open(
        os.path.join(helm_run_path, INDIVIDUAL_METRICS_FILENAME), "r"
    ) as metrics_file:
        instance_level_metrics_data = json.load(metrics_file)

    # Transform metrics
    aggregated_metrics: dict = transform_helm_metrics(
        aggregated_metrics_data,
        aggregation_method,
        target_split,
        metrics_map,
    )

    instance_level_metrics: list[dict] = []
    for instance in instance_level_metrics_data:
        instance_metrics = transform_helm_metrics(
            instance["stats"],
            aggregation_method,
            target_split,
            metrics_map,
        )
        instance_metrics["instance_id"] = instance["instance_id"]

        instance_level_metrics.append(instance_metrics)

    return {
        "aggregated_metrics": aggregated_metrics,
        "instance_level_metrics": instance_level_metrics,
    }

Replicate Healthcare AI Model Evaluator metrics function output:

In [ ]:
model_runs: dict[str, ModelRun] = {}
for helm_run_folder in os.listdir(MEDHELM_RUN_PATH):
    is_target_dataset = [
        helm_run_folder.startswith(candidate) for candidate in TARGET_DATASETS
    ]
    if not any(is_target_dataset):
        continue

    helm_run_path = os.path.join(MEDHELM_RUN_PATH, helm_run_folder)

    try:
        print(f"Processing {helm_run_folder}")
        model_run: ModelRun = process_helm_run(helm_run_path)
        metrics_results = process_helm_metrics(helm_run_path)
    except Exception:
        continue

    medbench_metrics_results = {
        "original_run": {
            "metrics_type": "helm",
            "model_run": model_run.to_json(),
        },
        "metrics_results": metrics_results,
        "processed_at": datetime.now(timezone.utc).isoformat(),
    }

    with open(
        os.path.join(OUTPUT_PATH, f"{helm_run_folder}.json"), "w+"
    ) as metrics_results_file:
        json.dump(medbench_metrics_results, metrics_results_file, indent=2)

Processing medication_qa:model=qwen_qwen2.5-7b-instruct
Processing medbullets:model=openai_gpt-4o,model_deployment=stanfordhealthcare_gpt-4o-2024-05-13
Processing race_based_med:model=google_gemini-1.5-pro-001,model_deployment=stanfordhealthcare_gemini-1.5-pro
Processing medbullets:model=meta_llama-3.3-70b-instruct,model_deployment=stanfordhealthcare_llama-3.3-70b-instruct
Processing head_qa:language=en,category=None,model=qwen_qwen2.5-7b-instruct
Processing race_based_med:model=meta_llama-3.3-70b-instruct,model_deployment=stanfordhealthcare_llama-3.3-70b-instruct
Processing ehr_sql:model=openai_gpt-4o,model_deployment=stanfordhealthcare_gpt-4o-2024-05-13
Processing aci_bench:model=openai_gpt-4o-mini,model_deployment=stanfordhealthcare_gpt-4o-mini-2024-07-18
Processing medication_qa:model=google_gemini-1.5-pro-001,model_deployment=stanfordhealthcare_gemini-1.5-pro
Processing medbullets:model=google_gemini-1.5-pro-001,model_deployment=stanfordhealthcare_gemini-1.5-pro
Processing pubmed_

## Transform data for Arena

In [15]:
import json

import pandas as pd

from medbench.datasets import CORRECT_TAG
from medbench.models import ModelRun

Point to where the transformed data from MedHelm is

In [18]:
# Set it to OUTPUT_PATH/func_output
MEDHELM_METRICS_PATH: str = "../../data/medhelm/medbench_connection/medhelm-mixed/func_output"

MEDBENCH_ARENA_DATASET_TARGET_PATH: str = "../../data/medhelm/medbench_connection/medhelm-mixed/arena/datasets"
MEDBENCH_ARENA_METRICS_TARGET_PATH: str = "../../data/medhelm/medbench_connection/medhelm-mixed/arena/metrics"

if not os.path.exists(MEDHELM_METRICS_PATH):
    os.makedirs(MEDHELM_METRICS_PATH)
if not os.path.exists(MEDBENCH_ARENA_DATASET_TARGET_PATH):
    os.makedirs(MEDBENCH_ARENA_DATASET_TARGET_PATH)
if not os.path.exists(MEDBENCH_ARENA_METRICS_TARGET_PATH):
    os.makedirs(MEDBENCH_ARENA_METRICS_TARGET_PATH)

Helper function for getting dataset names:

In [19]:
def get_dataset_id(medhelm_run_id: str, dataset_metadata: list[str] = ["category", "subset"]) -> str:
    run_id_split = medhelm_run_id.split(":")
    dataset_name = run_id_split[0]
    metadata_str = ""
    if len(run_id_split) == 2:
        dataset_name, metadata_str = run_id_split
    elif len(run_id_split) > 2:
        dataset_name = run_id_split[0]
        metadata_str = ",".join(run_id_split[1:])

    dataset_name_split = dataset_name.split(",")

    dataset_id = dataset_name_split[0]
    if len(dataset_name_split) > 1:
        metadata_str += "," + ",".join(dataset_name_split[1:])
    
    metadata = metadata_str.split(",")
    # Sort the metadata to ensure consistent ordering
    metadata.sort()
    for data in metadata:
        kv_split = data.split("=")
        if len(kv_split) != 2:
            continue
        key, value = kv_split 
        if key in dataset_metadata:
            dataset_id += f"-{value}"
    
    return dataset_id

Load transformed `ModelRun` objects and transform data into `jsonl` format

In [49]:
# Map file names to model names
clinical_tasks_metrics: dict = {}
clinical_tasks_data: dict[str, list] = {}
for model_run_filename in os.listdir(MEDHELM_METRICS_PATH):
    model_run_path = os.path.join(MEDHELM_METRICS_PATH, model_run_filename)
    if os.path.isdir(model_run_path):
        continue

    metrics_data: dict
    with open(model_run_path, "r") as f:
        metrics_data = json.load(f)

    model_run = ModelRun.from_json(metrics_data["original_run"]["model_run"])

    clinical_task = get_dataset_id(model_run_filename)
    if clinical_task.startswith("head_qa"):
        clinical_task = "head_qa"

    # Store metrics
    if clinical_task not in clinical_tasks_metrics:
        clinical_tasks_metrics[clinical_task] = {}
    clinical_tasks_metrics[clinical_task][model_run.model.name] = metrics_data[
        "metrics_results"
    ]["aggregated_metrics"]

    # Store run data
    if clinical_task not in clinical_tasks_data:
        clinical_tasks_data[clinical_task] = {}

    arena_dataset_data: list[dict] = []
    # Sort instances by their id and results by their input_id before zipping
    instances_sorted = sorted(model_run.dataset.instances, key=lambda inst: inst.id)
    results_sorted = sorted(model_run.results, key=lambda res: res.input_id)

    if len(instances_sorted) != len(results_sorted):
        raise ValueError(f"Length mismatch between instances ({len(instances_sorted)}) and results ({len(results_sorted)}) for {clinical_task}")

    for input, output in zip(instances_sorted, results_sorted):
        if input.id != output.input_id:
            raise ValueError(f"ID mismatch after sorting: instance {input.id} != result {output.input_id} in {clinical_task}")

        correct_answer: str = None
        for ref in input.references:
            if CORRECT_TAG not in ref.tags:
                continue

            # This assume a single output was given
            ref_content = ref.output.content[0]
            if ref_content.metadata and "multiple_choice_letter" in ref_content.metadata:
                correct_answer = ref_content.metadata["multiple_choice_letter"]
            else:
                # This supports multiple outputs.
                correct_answer = ref.output.get_text()
            break

        if correct_answer is None:
            raise ValueError(
                f"Could not find correct answer for instance {input.id} in dataset {clinical_task}"
            )

        arena_dataset_data.append(
            {
                "id": input.id,
                "input": input.input.get_text(),
                "output": output.completions.get_text(),
                "references": correct_answer,
            }
        )
    
    # print(f"{clinical_task} {model_run.model.name} {len(arena_dataset_data)} - {model_run_filename}")
    clinical_tasks_data[clinical_task][model_run.model.name] = arena_dataset_data

Save data:

In [71]:
# Save the metrics data
print("Saving metrics data")
for clinical_task in clinical_tasks_metrics:
    with open(
        os.path.join(MEDBENCH_ARENA_METRICS_TARGET_PATH, f"{clinical_task}.json"),
        "w+",
    ) as metrics_results_file:
        json.dump(
            {
                "metrics": clinical_tasks_metrics[clinical_task],
                "clinical_task": clinical_task,
            },
            metrics_results_file,
            indent=2,
        )

# Save the dataset data
print("Saving dataset data")
for clinical_task in clinical_tasks_data:
    print(f"\tSaving {clinical_task} dataset data")
    merged_df: pd.DataFrame = None
    for model_name in clinical_tasks_data[clinical_task]:
        model_df = pd.DataFrame(clinical_tasks_data[clinical_task][model_name])
        model_df.rename(
            columns={
                "output": model_name,
            },
            inplace=True,
        )

        if merged_df is None:
            merged_df = model_df
            continue

        print("\t\tMerging with", model_name, model_df.columns)
        # # Check if model_df["input"] and model_df["references"] are the same as in merged_df
        # if not model_df["input"].equals(merged_df["input"]):
        #     print("\t\t\tWarning: input columns do not match between models, skipping input column merge")
        # if not model_df["references"].equals(merged_df["references"]):
        #     print("\t\t\tWarning: references columns do not match between models, skipping references column merge")

        # model_df.drop(columns=["input", "references"], inplace=True)
        model_df.drop(columns=["id", "references"], inplace=True)
        merged_df = pd.merge(
            merged_df,
            model_df,
            how="outer",
            # NOTE: We merge on "input" because MedHelm ids may not be consistent across (version) runs.
            on=["input"],
            # suffixes=(None, f"_{model_name}"),
        )

    print("\n\t\tStats:")
    print(f"\t\t- Merged {len(merged_df)} rows for {clinical_task} dataset")
    
    # Log columns with and without NaN values
    if merged_df.isna().any().any():
        na_counts = merged_df.isna().sum()
        cols_with_na_counts = [f"- {col} ({int(cnt)})" for col, cnt in na_counts.items() if cnt > 0]
        cols_without_na = [col for col, cnt in na_counts.items() if cnt == 0]

        print(f"\t\t- Columns with NaN ({len(cols_with_na_counts)}): {', '.join(cols_with_na_counts) if cols_with_na_counts else 'None'}")
        print(f"\t\t- Columns without NaN ({len(cols_without_na)}): {', '.join(cols_without_na) if cols_without_na else 'None'}\n")

    merged_df.drop(columns=["id"], inplace=True)
    merged_df.to_json(
        os.path.join(MEDBENCH_ARENA_DATASET_TARGET_PATH, f"{clinical_task}.jsonl"),
        orient="records",
        lines=True,
    )

Saving metrics data
Saving dataset data
	Saving ehr_sql dataset data
		Merging with google/gemini-1.5-pro-001 Index(['id', 'input', 'google/gemini-1.5-pro-001', 'references'], dtype='object')
		Merging with google/gemini-2.0-flash-001 Index(['id', 'input', 'google/gemini-2.0-flash-001', 'references'], dtype='object')
		Merging with meta/llama-3.3-70b-instruct Index(['id', 'input', 'meta/llama-3.3-70b-instruct', 'references'], dtype='object')
		Merging with anthropic/claude-3-7-sonnet-20250219 Index(['id', 'input', 'anthropic/claude-3-7-sonnet-20250219', 'references'], dtype='object')
		Merging with qwen/qwen2.5-7b-instruct Index(['id', 'input', 'qwen/qwen2.5-7b-instruct', 'references'], dtype='object')
		Merging with openai/gpt-4o Index(['id', 'input', 'openai/gpt-4o', 'references'], dtype='object')
		Merging with anthropic/claude-3-5-sonnet-20241022 Index(['id', 'input', 'anthropic/claude-3-5-sonnet-20241022', 'references'], dtype='object')
		Merging with openai/gpt-4o-mini Index(['id

Log clinical tasks with missing values

In [63]:
# clinical_task = "aci_bench"
for clinical_task in clinical_tasks_data:
    merged_df: pd.DataFrame = None
    for model_name in clinical_tasks_data[clinical_task]:
        model_df = pd.DataFrame(clinical_tasks_data[clinical_task][model_name])
        model_df.rename(
            columns={
                "output": model_name,
            },
            inplace=True,
        )

        if merged_df is None:
            merged_df = model_df
            continue

        model_df.drop(columns=["input", "references"], inplace=True)
        merged_df = pd.merge(
            merged_df,
            model_df,
            how="outer",
            on=["id"],
        )

    # Print column names that contain na values, along with the number of na values
    print(f"Missing values in {clinical_task}")
    for column in merged_df.columns:
        if merged_df[column].isna().any():
            print(f"\t{column}: {merged_df[column].isna().sum()}")
        else:
            print(f"\t{column}: No missing values")

    print()


Missing values in ehr_sql
	id: No missing values
	input: No missing values
	microsoft/phi-3.5-mini-instruct: No missing values
	references: No missing values
	google/gemini-1.5-pro-001: No missing values
	google/gemini-2.0-flash-001: No missing values
	meta/llama-3.3-70b-instruct: No missing values
	anthropic/claude-3-7-sonnet-20250219: No missing values
	qwen/qwen2.5-7b-instruct: No missing values
	openai/gpt-4o: No missing values
	anthropic/claude-3-5-sonnet-20241022: No missing values
	openai/gpt-4o-mini: No missing values
	openai/o3-mini-2025-01-31: No missing values
	deepseek-ai/deepseek-r1-hide-reasoning: No missing values

Missing values in medec
	id: No missing values
	input: No missing values
	meta/llama-3.3-70b-instruct: No missing values
	references: No missing values
	microsoft/phi-3.5-mini-instruct: No missing values
	anthropic/claude-3-7-sonnet-20250219: No missing values
	google/gemini-2.0-flash-001: No missing values
	openai/gpt-4o: No missing values
	openai/o3-mini-202